# Getting started with the Playground API
This Notebook shows how to connect to the Intelligence Playground Python Client API and do amazing stuff!

In [2]:
# Install some extra python module if needed...
#!pip install -r requirements.txt

In [2]:
# Standard imports
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
import mercantile
import requests
import csv
import uuid
import json
import geohash2
import uuid
import warnings
import geojson
import time
warnings.filterwarnings("ignore")

from shapely.geometry import mapping, shape

# Widgets imports
from ipyleaflet import Map, TileLayer, Polygon
import ipywidgets as widgets
from IPython.core.display import HTML

## Connecting to the API

To connect to the API, you need to retrieve your API_KEYS from the OneAtlas website. Follow the simple steps below:

1. Visit this URL: https://data.api.oneatlas.airbus.com/api-keys
2. Click the **Create and API key** button
3. Enter a description for your API_KEY (e.g. Playground Keys)
4. Store the file in the same folder than this notebook and name it **api_key.txt**

Make sure to keep your **api_key.txt** safe! Do not include it in a public github repository for example :-)

The following script will then use this **api_key.txt** file to generate an ACCESS_TOKEN. We will store this ACCESS_TOKEN in HEADERS that we will send with each requests. The ACCESS_TOKEN has a timeout so we will create a function that renew the ACCESS_TOKEN when half of the timeout has expired. 

## Check connection

In [5]:
# Create a PlaygroundClient object
from playgroundclient import PlaygroundClient
play = PlaygroundClient()

In [6]:
# Logged in user
user = play.get_logged_user()
print("Logged as user: {} {}".format(user['firstname'], user['lastname']))

Logged as user: Jean-Francois Faudi


## Some useful functions

In [7]:
def print_json(parsed):
    print(json.dumps(parsed, indent=4, sort_keys=True))
    
def getBounds(coords):
    xmax = ymax = -np.Infinity
    xmin = ymin = np.Infinity
    for v in coords:
        xmax = xmax if xmax > v[0] else v[0]
        ymax = ymax if ymax > v[1] else v[1]
        xmin = xmin if xmin < v[0] else v[0]
        ymin = ymin if ymin < v[1] else v[1]
    return (xmin, ymin, xmax, ymax)


## Get available projects for this user

In [8]:
p_database = {}
p_label = []
r = play.get_projects()
#print(r)
for project in r:
    #print(process)
    project_id = project['project_id']
    p_label.append((project['name'], project_id))
    p_database[project_id] = project
    
p_label.sort() 
p_w = widgets.Dropdown(
    options=p_label,
    description='Project:',
    disabled=False
)
display(p_w)

Dropdown(description='Project:', options=(('Airbus NA', 'e84b4d54-4c99-4527-8be7-927a11a73897'), ('Airbus NA -…

## Get available datasets in this project

In [9]:
projectId = p_w.value

d_database = {}
d_label = []
r = play.get_datasets(projectId)
#print_json(r)
for dataset in r['datasets']:
    #print_json(dataset)
    dataset_id = dataset['datasetId']
    d_label.append((dataset['name'], dataset_id))
    d_database[dataset_id] = dataset
    
d_label.sort() 
d_w = widgets.Dropdown(
    options=d_label,
    description='Dataset:',
    disabled=False
)
display(d_w)

Dropdown(description='Dataset:', options=(('Airports demo', 'b5b41429-3399-4599-953d-3fbbb53d9488'), ('GC', '4…

In [10]:
# Dataset
datasetId = d_w.value

# display information about this dataset
r = play.get_dataset(datasetId)
#print_json(r)
print("This dataset contains {} zones.".format(r['zones']))
print("This dataset contains {} records.".format(r['records']))

This dataset contains 10 zones.
This dataset contains 1568 records.


In [11]:
z_database = {}
z_label = []
zones = play.get_zones_in_dataset(datasetId)
#print_json(zones)
for zone in zones:
    zone_id = zone['zone_id']
    z_label.append((zone['name'], zone_id))
    z_database[zone_id] = zone
    
z_label.sort() 
z_w = widgets.Dropdown(
    options=z_label,
    description='Zones:',
    disabled=False
)
display(z_w)

Dropdown(description='Zones:', options=(('Nouvelle-Aquitaine, France', '3951a03c-60c2-4ff9-977d-eb038837c51e')…

## Get zone bounding box

In [14]:
zone_id = z_w.value
print("Zone_ID={}".format(zone_id))
print_json(z_database[zone_id])
print_json(z_database[zone_id]['geom']['coordinates'][0])

Zone_ID=99555067-be79-4ea4-8a9d-ed11076bca55
{
    "created_at": "2019-06-04T13:38:08.549629Z",
    "dataset_id": "b5b41429-3399-4599-953d-3fbbb53d9488",
    "description": null,
    "geom": {
        "coordinates": [
            [
                [
                    -0.751896,
                    44.818977
                ],
                [
                    -0.727501,
                    44.82763
                ],
                [
                    -0.751896,
                    44.829865
                ],
                [
                    -0.751896,
                    44.844888
                ],
                [
                    -0.716858,
                    44.834448
                ],
                [
                    -0.648537,
                    44.884093
                ],
                [
                    -0.611801,
                    44.867063
                ],
                [
                    -0.698662,
                    44.830552
    

## Get records in zone

In [15]:
bounds = getBounds(z_database[zone_id]['geom']['coordinates'][0])
bbox = ",".join(map(str, bounds))
r = play.get_records_count_in_zone(zone_id, bbox)
#print_json(r)
print("This zone contains {} records.".format(r['count']))

This zone contains 322 records.
